In [ ]:
import os
import SimpleITK as sitk
import math
import glob
import pandas as pd
import random
import time
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import PIL
import pydicom

In [ ]:
#读取nrrd的CT并转换成numpy并保存
ct_path='RT222335\CT.nrrd'#路径修改处
ct=sitk.ReadImage(ct_path)
ct_array=sitk.GetArrayFromImage(ct)
#保存为{}_ct.npy
i=ct_path[0:8]
savepath='input\CT\{}_ct.npy'.format(i)
np.save(savepath,ct_array)
ct_array.shape

In [ ]:
#开始structure的提取
def drawslince(image, msk, contor):
    # size=image.GetSize()
    ctor = []
    for point in contor:
        temppoint = image.TransformPhysicalPointToIndex(point)
        ctor.append((temppoint[0], temppoint[1]))
    ctor = np.array(ctor).reshape(-1, 1, 2)
    # slince=np.zeros((size[0],size[1]))
    msk = cv2.drawContours(msk, [ctor], -1, 1, thickness=cv2.FILLED)

    return msk, temppoint[2]

from dicompylercore import dicomparser
########################
# 从rtstruct中提取ROI，ROI名字不分大小写包含tg_ctor_name,为mask
# mask2nii(image,mskpath,tg_ctor_name,savepath)
# image : 原始图像SimpleITK.SimpleITK.Image类型文件
# mskpath: RTstruct文件目录
# tg_ctor_name: mask roi 名字
# savepath保存路径
#########################
def mask2nii(image, mskpath, tg_ctor_name, savepath):
    tg_ctor_name = tg_ctor_name.upper()
    rtss = dicomparser.DicomParser(mskpath)
    rois = rtss.GetStructures()
    roikey = [roi_id for roi_id in rois.keys() if rois[roi_id]['name'].upper() == tg_ctor_name]

    if len(roikey) == 0:
        return None
    ########read mask

    size = image.GetSize()  # x, y, z
    imagemask = np.zeros(size)
    roi = rtss.GetStructureCoordinates(roikey[0])
    for slicer_i in roi.keys():
        msk = np.zeros((size[0], size[1]))
        maskcount = len(roi[slicer_i])

        for i in range(maskcount):
            tempcontor = roi[slicer_i][i]['data']
            msk, slicenum = drawslince(image, msk, tempcontor)
        #     sigalcontor=[image.TransformPhysicalPointToIndex(point) for point in tempcontor] #x,y,z
        # imagemask=cv2.drawContours(image,sigalcontor,-1,255)
        # # if maskcount==1:
        #     plt.figure()
        #     plt.imshow(msk,cmap='gray')
        imagemask[:, :, slicenum] = msk
    imagemask = imagemask.transpose((2, 0, 1))
    mask = sitk.GetImageFromArray(imagemask)
    mask.SetOrigin(image.GetOrigin())
    mask.SetDirection(image.GetDirection())
    mask.SetSpacing(image.GetSpacing())
    # if not savepath == '':
    #mask转换成array
    # mask=sitk.ReadImage(mask)
    # mask_array=sitk.GetArrayFromImage(mask)
    # np.save(savepath,mask_array)
    sitk.WriteImage(mask, savepath)

    return mask

In [ ]:
#开始structure的提取，是nii格式（每个单独修改）
image_path='RT222335\CT.nrrd'
image=sitk.ReadImage(image_path)
maskpath='RT222335\RS\95266961_StrctrSets.dcm'
tg_ctor_name='PTV'#########################################危及器官名称修改处
i=image_path[0:8]
savepath='input\PTV\{}_ptv.nii'.format(i)
mask2nii(image, maskpath, tg_ctor_name, savepath)

#转换成数组numpy格式并保存
mask=sitk.ReadImage(savepath)
mask_array=sitk.GetArrayFromImage(mask)
array_path='input\PTV\{}_ptv.npy'.format(i)########危及器官名称修改
np.save(array_path,mask_array)